In [1]:
import torch
from transformers import pipeline


device = "cuda:0" if torch.cuda.is_available() else "cpu"
pipe = pipeline(
    "automatic-speech-recognition", model="openai/whisper-base", device=device
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
from datasets import load_dataset

# load audio sample in italian
dataset = load_dataset("facebook/voxpopuli", "it", split="validation", streaming=True)

/Users/enyiomaosondu/personal/final-year-project/env/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for facebook/voxpopuli contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/facebook/voxpopuli
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [3]:
sample = next(iter(dataset))

In [4]:
from IPython.display import Audio

Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])

In [5]:
def translate(audio):
    outputs = pipe(audio, max_new_tokens=256, generate_kwargs={"task": "translate"})
    return outputs["text"]


def transcribe(audio, language="en"):
    outputs = pipe(audio, max_new_tokens=256, generate_kwargs={"task": "transcribe", "language": language})
    return outputs["text"]

In [6]:
translate(sample["audio"].copy())

' psychological and social. I think that it is a very important step in the construction of a juridical space of freedom, circulation and protection of rights.'

In [7]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")

model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

In [8]:
from datasets import load_dataset

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

In [9]:
def synthesise(text):
    inputs = processor(text=text, return_tensors="pt")
    speech = model.generate_speech(
        inputs["input_ids"].to(device), speaker_embeddings.to(device), vocoder=vocoder
    )
    return speech.cpu()

In [10]:
speech = synthesise("Hey there! This is a test!")

Audio(speech, rate=16000)

In [11]:
import numpy as np

target_dtype = np.int16
max_range = np.iinfo(target_dtype).max


def speech_to_speech_translation(audio, target_language="es"):
    translated_text = transcribe(audio, target_language)
    print(translated_text)
    synthesised_speech = synthesise(translated_text)
    synthesised_speech = (synthesised_speech.numpy() * max_range).astype(np.int16)
    return 16000, synthesised_speech

In [12]:
sampling_rate, synthesised_speech = speech_to_speech_translation(sample["audio"].copy(), "en")

Audio(synthesised_speech, rate=sampling_rate)

 psychological and social. I think that if a step forward is important in the construction of a juridical space of freedom, circulation and protection of rights,


In [ ]:
import gradio as gr

demo = gr.Blocks()

mic_translate = gr.Interface(
    fn=speech_to_speech_translation,
    inputs=gr.Audio(sources=["microphone"], type="filepath"),
    outputs=gr.Audio(label="Generated Speech", type="numpy"),
)

file_translate = gr.Interface(
    fn=speech_to_speech_translation,
    inputs=gr.Audio(sources=["upload"], type="filepath"),
    outputs=gr.Audio(label="Generated Speech", type="numpy"),
)

with demo:
    gr.TabbedInterface([mic_translate, file_translate], ["Microphone", "Audio File"])

demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


 Pist.
 Es una sometimes en inglés.
 Esto es el texto sentado en inglés.
